## Deep Learning for Mortality Prediction (DLMP)

### Results currently in this analysis 

- val loss for combined model
- val loss for states only with combined data
- val loss for countries only with combined data 
- val loss for combined data with Lee-Carter 
- missing important information in this file: loss for models with states and countries alone - is this in some other file? If not it maybe should be?

### Goals of this analysis
 - assess performance of the deep learning model for US states and countries using a combined dataset
 - compare performance to baselines (Lee-Carter)

### Possible future questions worth asking 
 - would it also improve relative to coherant LC extensions? 
 - could do a more gradual increase in sample size (compare performance adding a tenth of the states at a time) 

### Import packages 

In [185]:
import tensorflow as tf
import csv
import numpy as np
import pandas as pd
import os as os
import matplotlib.pyplot as plt
tfkl = tf.keras.layers

### Prepare data

In [186]:
# loading in USMDB data
data = []
ages = []
states = []
genders = []

with open("../data/usmdb/usmdb.csv", "r") as file:
    reader = csv.reader(file,delimiter=',')
    for row_index, row in enumerate(reader):
        if row_index == 0:
            print(row)
        if row_index >= 1:
            state, gender, year, age, rate = row
            year = int(year)
            try:
                age = int(age)
            except:
                age = -1
            if state not in states:
                states.append(state)
            state = states.index(state)
            if gender not in genders:
                genders.append(gender)
            gender = genders.index(gender)
            try:
                rate = float(rate)
            except:
                rate = -1
            if rate > 1:
                rate = 1
            # get rid of years, ages, not in health data and other cleaning
            if age != -1 and rate != -1 and age <= 99:
                data.append([state, gender, year, age, rate])

state_data = np.array(data)

['PopName', 'Sex', 'Year', 'Age', 'mx']


In [187]:
# loading in HMD data
data = []
ages = []
countries = []
genders = []

with open("../data/hmd.csv", "r") as file:
    reader = csv.reader(file,delimiter=",")
    for row_index, row in enumerate(reader):
        if row_index == 0:
            print(row)
        if row_index >= 1:
            country, gender, year, age, rate = row
            year = int(year)
            try:
                age = int(age)
            except:
                age = -1
            if country not in countries:
                countries.append(country)
            country = countries.index(country)
            if gender not in genders:
                genders.append(gender)
            gender = genders.index(gender)
            try:
                rate = float(rate)
            except:
                rate = -1
            if rate > 1:
                rate = 1
            if age != -1 and rate != -1 and age <= 99:
                data.append([country, gender, year, age, rate])

country_data = np.array(data)

['Country', 'Gender', 'Year', 'Age', 'Mortality_rate']


### Train Seperate DL Models for Country and State Data

#### State Model

In [188]:
# training and test sets 
training_index = np.logical_and(state_data[:, 2] >= 1959, state_data[:, 2] <= 2005)
state_training = state_data[training_index, :]

test_index = np.logical_and(state_data[:, 2] > 2005, state_data[:, 2] <= 2015)
state_test = state_data[test_index, :]

final_test_index = np.logical_and(state_data[:, 2] > 2015, state_data[:, 2] <= 2019)
state_final_test = state_data[final_test_index, :]

In [189]:
state_training = tf.convert_to_tensor(state_training)
state_test = tf.convert_to_tensor(state_test)
state_final_test = tf.convert_to_tensor(state_final_test)
# cast tensor to type float32
state_training = tf.cast(state_training, tf.float32)
state_test = tf.cast(state_test, tf.float32)
state_final_test = tf.cast(state_final_test, tf.float32)

num_train = state_training.shape[0]
num_test = state_test.shape[0]
num_final = state_final_test.shape[0]

In [190]:
# define function to fetch and process data entries from training or test data (used in all iterations below)
def get_data(index, training_data, test_data, mode):
    if mode == "train":
        # randomly selects index from training data between 0 and num_train
        rand_index = tf.random.uniform([],minval=0, maxval=num_train, dtype=tf.int32) 
        entry = training_data[rand_index, :]
    elif mode == "not_random":
        # selects specified index from test data 
        entry = test_data[index, :]
    else: 
        # for any other value of mode, randomly selects index from test
        rand_index = tf.random.uniform([],minval=0, maxval=num_test, dtype=tf.int32)
        entry = test_data[rand_index, :]
    geography, gender, year, age, rate = entry[0], entry[1], entry[2], entry[3], entry[4]
    year = (year - 1998)/21
    age = tf.cast(age, tf.int32)
    geography = tf.cast(geography, tf.int32)
    gender = tf.cast(gender, tf.int32)
    year = tf.reshape(year, [1])
    age = tf.reshape(age, [1])
    geography = tf.reshape(geography, [1])
    gender = tf.reshape(gender, [1])
    rate = tf.reshape(rate, [1])
    return (year, age, geography, gender), rate

In [191]:
# use get_data function to set up training and test tensorflow datasets 
dataset_train = tf.data.Dataset.from_tensor_slices(np.arange(10000))
dataset_train = dataset_train.repeat()
dataset_train = dataset_train.map(lambda x: get_data(x, state_training, state_test, mode="train"), num_parallel_calls=4)
dataset_train = dataset_train.batch(256)
state_dataset_train = dataset_train.prefetch(buffer_size=512)

dataset_test = tf.data.Dataset.from_tensor_slices(np.arange(10000))
dataset_test = dataset_test.repeat()
dataset_test = dataset_test.map(lambda x: get_data(x, state_training, state_test, mode="test"), num_parallel_calls=4)
dataset_test = dataset_test.batch(256)
state_dataset_test = dataset_test.prefetch(buffer_size=512)

dataset_test2 = tf.data.Dataset.from_tensor_slices(np.arange(68000))
dataset_test2 = dataset_test2.map(lambda x: get_data(x, state_training, state_test, mode="not_random"), num_parallel_calls=4)
dataset_test2 = dataset_test2.batch(256)
state_dataset_test2 = dataset_test2.prefetch(buffer_size=512)

dataset_final = tf.data.Dataset.from_tensor_slices(np.arange(10000))
dataset_final = dataset_final.repeat()
dataset_final = dataset_final.map(lambda x: get_data(x, state_training, state_test, mode="test"), num_parallel_calls=4)
dataset_final = dataset_final.batch(256)
state_dataset_final = dataset_final.prefetch(buffer_size=512)

In [157]:
def create_model(geo_dim):
    # defining inputs 
    year = tfkl.Input(shape=(1,), dtype='float32', name='Year')
    age =  tfkl.Input(shape=(1,), dtype='int32', name='Age')
    geography = tfkl.Input(shape=(1,), dtype='int32', name='Geography')
    gender = tfkl.Input(shape=(1,), dtype='int32', name='Gender')

    # defining embedding layers 
    age_embed = tfkl.Embedding(input_dim=100, output_dim=5, name='Age_embed')(age)
    age_embed = tfkl.Flatten()(age_embed)

    gender_embed = tfkl.Embedding(input_dim=2, output_dim=5, name='Gender_embed')(gender)
    gender_embed = tfkl.Flatten()(gender_embed)

    geography_embed = tfkl.Embedding(input_dim=geo_dim, output_dim=5, name='Geography_embed')(geography)
    geography_embed = tfkl.Flatten()(geography_embed)

    # create feature vector that concatenates all inputs 
    x = tfkl.Concatenate()([year, age_embed, gender_embed, geography_embed])
    x1 = x

    # setting up middle layers 
    x = tfkl.Dense(128, activation='tanh')(x)
    x = tfkl.BatchNormalization()(x)
    x = tfkl.Dropout(0.05)(x)

    x = tfkl.Dense(128, activation='tanh')(x)
    x = tfkl.BatchNormalization()(x)
    x = tfkl.Dropout(0.05)(x)

    x = tfkl.Dense(128, activation='tanh')(x)
    x = tfkl.BatchNormalization()(x)
    x = tfkl.Dropout(0.05)(x)

    x = tfkl.Dense(128, activation='tanh')(x)
    x = tfkl.BatchNormalization()(x)
    x = tfkl.Dropout(0.05)(x)

    # setting up output layer 
    x = tfkl.Concatenate()([x1, x])
    x = tfkl.Dense(128, activation='tanh')(x)
    x = tfkl.BatchNormalization()(x)
    x = tfkl.Dropout(0.05)(x)
    x = tfkl.Dense(1, activation='sigmoid', name='final')(x)

    # creating the model 
    model = tf.keras.Model(inputs=[year, age, geography, gender], outputs=[x])

    # compiling the model
    model.compile(loss='mse', optimizer='adam')

    return model

In [158]:
def run_deep_model(dataset_train, dataset_test, geo_dim):
    
    model = create_model(geo_dim)

    callbacks = [tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.25, patience=3, verbose=0, mode="auto", 
                                                    min_delta=1e-8, cooldown=0, min_lr=0.0)]
    history = model.fit(dataset_train, steps_per_epoch=1000, validation_data=dataset_test, validation_steps=500, 
                        epochs=30, verbose=2, callbacks=callbacks)

    loss_info = {
        'train_mse': history.history['loss'][-1],
        'val_mse': history.history['val_loss'][-1]
    }

    tf.keras.backend.clear_session()

    return model, loss_info


In [159]:
# get the proper geography input dimension for model set up 
unique_vals = tf.unique(state_training[:, 0]).y
state_geo_dim = np.array(tf.size(unique_vals)).item()
print(geo_dim)

50


In [160]:
model_state, loss_info_state = run_deep_model(state_dataset_train, state_dataset_test, state_geo_dim)

Epoch 1/30
1000/1000 - 22s - 22ms/step - loss: 0.0140 - val_loss: 0.0010 - learning_rate: 0.0010
Epoch 2/30
1000/1000 - 16s - 16ms/step - loss: 6.5527e-04 - val_loss: 8.3409e-05 - learning_rate: 0.0010
Epoch 3/30
1000/1000 - 13s - 13ms/step - loss: 3.4390e-04 - val_loss: 5.4184e-04 - learning_rate: 0.0010
Epoch 4/30
1000/1000 - 14s - 14ms/step - loss: 2.4983e-04 - val_loss: 4.6172e-04 - learning_rate: 0.0010
Epoch 5/30
1000/1000 - 17s - 17ms/step - loss: 1.9296e-04 - val_loss: 7.4378e-05 - learning_rate: 0.0010
Epoch 6/30
1000/1000 - 15s - 15ms/step - loss: 1.6219e-04 - val_loss: 1.0699e-04 - learning_rate: 0.0010
Epoch 7/30
1000/1000 - 14s - 14ms/step - loss: 1.3653e-04 - val_loss: 4.3395e-05 - learning_rate: 0.0010
Epoch 8/30
1000/1000 - 14s - 14ms/step - loss: 1.1832e-04 - val_loss: 2.0244e-04 - learning_rate: 0.0010
Epoch 9/30
1000/1000 - 14s - 14ms/step - loss: 1.0922e-04 - val_loss: 6.2791e-05 - learning_rate: 0.0010
Epoch 10/30
1000/1000 - 16s - 16ms/step - loss: 1.0173e-04 - va

In [161]:
print(loss_info_state)

{'train_mse': 5.8527719374978915e-05, 'val_mse': 4.5659751776838675e-05}


#### Country Model

In [197]:
# training and test sets 
training_index = np.logical_and(country_data[:, 2] >= 1959, country_data[:, 2] <= 2005)
country_training = country_data[training_index, :]
print(country_training.shape)

test_index = np.logical_and(country_data[:, 2] > 2005, country_data[:, 2] <= 2015)
country_test = country_data[test_index, :]

final_test_index = np.logical_and(country_data[:, 2] > 2015, country_data[:, 2] <= 2019)
country_final_test = country_data[final_test_index, :]

(340794, 5)


In [198]:
country_training = tf.convert_to_tensor(country_training)
country_test = tf.convert_to_tensor(country_test)
country_final_test = tf.convert_to_tensor(country_final_test)
# cast tensor to type float32
country_training = tf.cast(country_training, tf.float32)
country_test = tf.cast(country_test, tf.float32)
country_final_test = tf.cast(country_final_test, tf.float32)

num_train = country_training.shape[0]
num_test = country_test.shape[0]
num_final = country_final_test.shape[0]

In [199]:
# use get_data function to set up training and test tensorflow datasets 
dataset_train = tf.data.Dataset.from_tensor_slices(np.arange(10000))
dataset_train = dataset_train.repeat()
dataset_train = dataset_train.map(lambda x: get_data(x, country_training, country_test, mode="train"), num_parallel_calls=4)
dataset_train = dataset_train.batch(256)
country_dataset_train = dataset_train.prefetch(buffer_size=512)

dataset_test = tf.data.Dataset.from_tensor_slices(np.arange(10000))
dataset_test = dataset_test.repeat()
dataset_test = dataset_test.map(lambda x: get_data(x, country_training, country_test, mode="test"), num_parallel_calls=4)
dataset_test = dataset_test.batch(256)
country_dataset_test = dataset_test.prefetch(buffer_size=512)

dataset_test2 = tf.data.Dataset.from_tensor_slices(np.arange(68000))
dataset_test2 = dataset_test2.map(lambda x: get_data(x, country_training, country_test, mode="not_random"), num_parallel_calls=4)
dataset_test2 = dataset_test2.batch(256)
country_dataset_test2 = dataset_test2.prefetch(buffer_size=512)

dataset_final = tf.data.Dataset.from_tensor_slices(np.arange(10000))
dataset_final = dataset_final.repeat()
dataset_final = dataset_final.map(lambda x: get_data(x, country_training, country_test, mode="test"), num_parallel_calls=4)
dataset_final = dataset_final.batch(256)
country_dataset_final = dataset_final.prefetch(buffer_size=512)

In [200]:
# get the proper geography input dimension for model set up 
unique_vals = tf.unique(country_training[:, 0]).y
country_geo_dim = np.array(tf.size(unique_vals)).item()
print(geo_dim)

38


In [ ]:
model_country, loss_info_country = run_deep_model(country_dataset_train, country_dataset_test, country_geo_dim)

In [ ]:
print(country_loss_info)

### Train Combined DL Model

In [173]:
# getting unique values for geographic location column 
country_data[:,0] = country_data[:,0] + 50

# dropping US
country_data = country_data[country_data[:,0] != 87]
countries.remove('USA')

# merge data
combined = np.vstack((state_data, country_data))

ValueError: list.remove(x): x not in list

In [174]:
training_index = np.logical_and(combined[:, 2] >= 1959, combined[:, 2] <= 2005)
combined_training = combined[training_index, :]
print(combined_training.shape)

test_index = np.logical_and(combined[:, 2] > 2005, combined[:, 2] <= 2015)
combined_test = combined[test_index, :]
print(combined_test.shape)

final_test_index = np.logical_and(combined[:, 2] > 2015, combined[:, 2] <= 2019)
combined_final_test = combined[final_test_index, :]
print(combined_final_test.shape)

(801394, 5)
(172000, 5)
(66400, 5)


In [175]:
# prepare training and test data 
combined_training = tf.convert_to_tensor(combined_training)
combined_test = tf.convert_to_tensor(combined_test)
combined_final_test = tf.convert_to_tensor(combined_final_test)

combined_training = tf.cast(combined_training, tf.float32)
combined_test = tf.cast(combined_test, tf.float32)
combined_final_test = tf.cast(combined_final_test, tf.float32)

num_train = combined_training.shape[0]
num_test = combined_test.shape[0]
num_final = combined_final_test.shape[0]

In [176]:
# use get_data function to set up training and test tensorflow datasets 
dataset_train = tf.data.Dataset.from_tensor_slices(np.arange(10000))
dataset_train = dataset_train.repeat()
dataset_train = dataset_train.map(lambda x: get_data(x, combined_training, combined_test, mode="train"), num_parallel_calls=4)
dataset_train = dataset_train.batch(256)
combined_dataset_train = dataset_train.prefetch(buffer_size=512)

dataset_test = tf.data.Dataset.from_tensor_slices(np.arange(10000))
dataset_test = dataset_test.repeat()
dataset_test = dataset_test.map(lambda x: get_data(x, combined_training, combined_test, mode="test"), num_parallel_calls=4)
dataset_test = dataset_test.batch(256)
combined_dataset_test = dataset_test.prefetch(buffer_size=512)

dataset_test2 = tf.data.Dataset.from_tensor_slices(np.arange(68000))
dataset_test2 = dataset_test2.map(lambda x: get_data(x, combined_training, combined_test, mode="not_random"), num_parallel_calls=4)
dataset_test2 = dataset_test2.batch(256)
combined_dataset_test2 = dataset_test2.prefetch(buffer_size=512)

dataset_final = tf.data.Dataset.from_tensor_slices(np.arange(10000))
dataset_final = dataset_final.repeat()
dataset_final = dataset_final.map(lambda x: get_data(x, combined_training, combined_test, mode="test"), num_parallel_calls=4)
dataset_final = dataset_final.batch(256)
combined_dataset_final = dataset_final.prefetch(buffer_size=512)

In [180]:
# get the proper geography input dimension for model set up 
unique_vals = tf.unique(combined_training[:, 0]).y
combined_geo_dim = np.array(tf.size(unique_vals)).item()
print(geo_dim)

87


In [181]:
model_combined, loss_info_combined = run_deep_model(combined_dataset_train, combined_dataset_test, combined_geo_dim)

Epoch 1/30
1000/1000 - 22s - 22ms/step - loss: 0.0142 - val_loss: 3.4241e-04 - learning_rate: 0.0010
Epoch 2/30
1000/1000 - 15s - 15ms/step - loss: 0.0012 - val_loss: 5.8397e-04 - learning_rate: 0.0010
Epoch 3/30
1000/1000 - 14s - 14ms/step - loss: 7.1917e-04 - val_loss: 6.5208e-04 - learning_rate: 0.0010
Epoch 4/30
1000/1000 - 12s - 12ms/step - loss: 5.6941e-04 - val_loss: 1.8711e-04 - learning_rate: 0.0010
Epoch 5/30
1000/1000 - 15s - 15ms/step - loss: 4.7100e-04 - val_loss: 2.0635e-04 - learning_rate: 0.0010
Epoch 6/30
1000/1000 - 16s - 16ms/step - loss: 4.3342e-04 - val_loss: 1.7389e-04 - learning_rate: 0.0010
Epoch 7/30
1000/1000 - 14s - 14ms/step - loss: 4.0262e-04 - val_loss: 2.2599e-04 - learning_rate: 0.0010
Epoch 8/30
1000/1000 - 14s - 14ms/step - loss: 3.8968e-04 - val_loss: 1.6464e-04 - learning_rate: 0.0010
Epoch 9/30
1000/1000 - 14s - 14ms/step - loss: 3.8240e-04 - val_loss: 1.5528e-04 - learning_rate: 0.0010
Epoch 10/30
1000/1000 - 14s - 14ms/step - loss: 3.7677e-04 - va

In [182]:
# print combined loss info (loss for states and countries using full dataset)
print(loss_info_combined)

{'train_mse': 0.0002989772183354944, 'val_mse': 0.0001273844827665016}


### MSE for states only from combined model

In [194]:
def get_data(index, training_data, test_data, mode):
    if mode == "train":
        # Randomly selects index from training data between 0 and num_train
        rand_index = tf.random.uniform([], minval=0, maxval=num_train, dtype=tf.int32) 
        entry = training_data[rand_index, :]
    elif mode == "not_random":
        # Selects specified index from test data 
        entry = test_data[index, :]
    else:  # Assuming mode="test" or any other value
        # For any other value of mode, randomly selects index from test
        rand_index = tf.random.uniform([], minval=0, maxval=num_test, dtype=tf.int32)
        entry = test_data[rand_index, :]

    geography, gender, year, age, rate = entry[0], entry[1], entry[2], entry[3], entry[4]

    # Normalization
    year = (year - 1998) / 21
    age = tf.cast(age, tf.int32)
    geography = tf.cast(geography, tf.int32)
    gender = tf.cast(gender, tf.int32)

    # Reshape as needed (batched input)
    features = (tf.reshape(year, [1]), tf.reshape(age, [1]), tf.reshape(geography, [1]), tf.reshape(gender, [1]))
    rate = tf.reshape(rate, [1])
    return features, rate

# Creating the test dataset without `repeat()` for prediction
dataset_test = tf.data.Dataset.from_tensor_slices(np.arange(10000))

# Properly use `map` to call `get_data`
dataset_test = dataset_test.map(lambda x: get_data(x, state_training, state_test, mode="not_random"), num_parallel_calls=4)

# Batch the dataset for efficient predictions
dataset_test = dataset_test.batch(256)

# Prefetch to improve performance
state_dataset_test = dataset_test.prefetch(buffer_size=tf.data.AUTOTUNE)

# Now, run the predictions
predictions = model_combined.predict(state_dataset_test)

40/40 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step


In [195]:
# get the true values from the test dataset
true_values = []
for _, rate in state_dataset_test:
    true_values.extend(rate.numpy())

# convert true_values to a numpy array
true_values = np.array(true_values)

# convert predictions to a numpy array if not already
predictions = np.array(predictions)

# compute MSE using TensorFlow
mse = np.mean((true_values - predictions)**2)
print("Mean Squared Error:", mse)

Mean Squared Error: 2.473827e-05


### MSE for countries only from combined model

In [201]:
# Creating the test dataset without `repeat()` for prediction
dataset_test = tf.data.Dataset.from_tensor_slices(np.arange(10000))

# Properly use `map` to call `get_data`
dataset_test = dataset_test.map(lambda x: get_data(x, country_training, country_test, mode="not_random"), num_parallel_calls=4)

# Batch the dataset for efficient predictions
dataset_test = dataset_test.batch(256)

# Prefetch to improve performance
country_dataset_test = dataset_test.prefetch(buffer_size=tf.data.AUTOTUNE)

# Now, run the predictions
predictions = model_combined.predict(country_dataset_test)

40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step


In [202]:
# Get the true values from the test dataset
true_values = []
for _, rate in country_dataset_test:
    true_values.extend(rate.numpy())

# Convert true_values to a numpy array
true_values = np.array(true_values)

# Convert predictions to a numpy array if not already
predictions = np.array(predictions)

# Compute MSE using TensorFlow
mse = tf.reduce_mean(tf.square(true_values - predictions)).numpy()
print("Mean Squared Error:", mse)

Mean Squared Error: 0.0005840145


2024-08-30 17:02:06.121582: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


### Train Lee-Carter model

In [127]:
# non-tensor train / test split (same years in training / test here as in method above)
training_index = np.logical_and(combined[:, 2] >= 1959, combined[:, 2] <= 2005)
training_data = combined[training_index, :]

test_index = np.logical_and(combined[:, 2] > 2005, combined[:, 2] <= 2015)
test_data = combined[test_index, :]

final_test_index = np.logical_and(combined[:, 2] > 2015, combined[:, 2] <= 2019)
final_test = combined[final_test_index, :]

[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16. 17.
 18. 19. 20. 21. 22. 23. 24. 25. 26. 27. 28. 29. 30. 31. 32. 33. 34. 35.
 36. 37. 38. 39. 40. 41. 42. 43. 44. 45. 46. 47. 48. 49. 50. 51. 52. 53.
 54. 55. 56. 57. 58. 59. 60. 61. 62. 63. 64. 65. 66. 67. 68. 69. 70. 71.
 72. 73. 74. 75. 76. 77. 78. 79. 80. 81. 82. 83. 84. 85. 86.]


In [45]:
# set up lee-carter function

def lee_carter(mx_matrix):
    """
    Run the Lee-Carter model on age-specific mortality data.
    
    Args:
        mx_matrix (numpy.ndarray): A 2D array of age-specific mortality rates. rows = age, columns = years
        
    Returns:
        tuple: A tuple containing the estimated parameters (ax, bx, kt) and the fitted mortality rates.
    """
    mx_matrix[mx_matrix <= 0] = 1e-9

    ax = np.mean(np.log(mx_matrix), axis=1)
    ax = ax.reshape(-1, 1) # reshape ax into column vector
    
    centered_mx = np.log(mx_matrix) - ax
    
    # SVD
    U, S, Vt = np.linalg.svd(centered_mx, full_matrices=False)

    # extract right and left singular vectors (bx and kt)
    bx = U[:, 0]
    kt = Vt[0, :]
    # print(kt)

    # normalize bx and kt 
    bx = bx / np.sum(bx)
    # print(np.mean(kt))
    kt = kt - np.mean(kt)
    # print(np.mean(kt))

    # estimate fitted mortality 
    fitted_mort = np.exp(ax + np.outer(bx, kt))

    return (ax, bx, kt), fitted_mort

In [115]:
# set up function to run multiple models on all years in training data

def lee_carter_geo_gender(data):

    geos = np.unique(data[:, 0])
    genders = np.unique(data[:, 1])

    results = {}

    for geo in geos:
        for gender in genders:
            mask = (data[:, 0] == geo) & (data[:, 1] == gender)
            geo_gender_data = data[mask]

            # extract ages and years
            years = np.unique(geo_gender_data[:, 2])
            ages = np.unique(geo_gender_data[:, 3])

            m_x = np.zeros((len(ages), len(years)))

            for i, age in enumerate(ages):
                for j, year in enumerate(years):
                    mask = (geo_gender_data[:, 3] == age) & (geo_gender_data[:, 2] == year)
                    #m_x[i,j] = geo_gender_data[mask, 4]
                    selected_data = geo_gender_data[mask, 4]

                    # Ensure we handle different cases for selected_data
                    if selected_data.size == 0:
                        # No data available for this age and year
                        m_x[i, j] = np.nan  # Assign NaN or some default value
                    elif selected_data.size == 1:
                        # Exactly one value, the expected case
                        m_x[i, j] = selected_data[0]
                    else:
                        # More than one value, choose an aggregation method
                        print("more than 1 value")
                        m_x[i, j] = np.mean(selected_data)  # Or use np.median, np.min, etc.

            # Debugging
             # Check for NaN or infinite values
            if np.isnan(m_x).any() or np.isinf(m_x).any():
                print(f"Skipping Geo: {geo}, Gender: {gender} due to NaN or infinite values in m_x")
                continue

            try:
                params, fitted_mort = lee_carter(m_x)
            except np.linalg.LinAlgError as e:
                print(f"SVD did not converge for Geo: {geo}, Gender: {gender}. Error: {str(e)}")
                continue

            params, fitted_mort = lee_carter(m_x)
    
            # Store the results for the current geo and gender
            results[(geo, gender)] = {
                'params': params,
                'fitted_mortality': fitted_mort
            }
    
    return results
            

In [116]:
def lee_carter_forecast(results, h, start_year, ages, drift=True):
    """
    Perform the forecasting step of the Lee-Carter method using a random walk with drift.
    
    Args:
        results (dict): A dictionary containing the estimated parameters (ax, bx, kt) for each state and gender combination.
        h (int): The number of future periods to forecast.
        start_year (int): The starting year of the forecast.
        ages (numpy.ndarray): A 1D array of ages corresponding to the rows of the mortality matrix.
        drift (bool, optional): Whether to include a drift term in the random walk. Default is True.
        
    Returns:
        numpy.ndarray: A 2D array with 5 columns representing state, gender, year, age, and forecasted mortality rate.
    """
    
    forecasts = []
    
    for geo, gender in results.keys():
        ax, bx, kt = results[(geo, gender)]['params']
        
        # Estimate the drift term
        if drift:
            drift_term = (kt[-1] - kt[0]) / (len(kt) - 1)
        else:
            drift_term = 0
        
        # Forecast future kt values using a random walk with drift
        kt_forecast = np.zeros(h)
        kt_forecast[0] = kt[-1]
        for i in range(1, h):
            kt_forecast[i] = kt_forecast[i-1] + drift_term + np.random.normal(0, 1)
        
        # Forecast future mortality rates
        ax_matrix = np.repeat(ax, h).reshape(-1, h)
        bx_matrix = np.repeat(bx, h).reshape(-1, h)
        kt_matrix = np.repeat(kt_forecast, len(ax)).reshape(h, -1).T
        mortality_forecast = np.exp(ax_matrix + bx_matrix * kt_matrix)

        # Create a 2D array with geo, gender, year, age, and forecasted mortality rate
        for i in range(h):
            year = start_year + i
            for j, age in enumerate(ages):
                forecasts.append([geo, gender, year, age, mortality_forecast[j, i]])

    # Convert forecasts to a NumPy array
    forecasts = np.array(forecasts)

    # Sort the forecasts array based on the first four columns
    sorted_indices = np.lexsort((forecasts[:, 3], forecasts[:, 2], forecasts[:, 1], forecasts[:, 0]))
    forecasts = forecasts[sorted_indices]

    
    return forecasts

In [128]:
def calculate_mse(forecasted_rates, actual_rates):
    """
    Calculate the Mean Squared Error (MSE) between the forecasted and actual mortality rates.
    
    Args:
        forecasted_rates (numpy.ndarray): A 2D array with 5 columns representing state, gender, year, age, and forecasted mortality rate.
        actual_rates (numpy.ndarray): A 2D array with 5 columns representing state, gender, year, age, and actual mortality rate.
        
    Returns:
        float: The Mean Squared Error (MSE) between the forecasted and actual mortality rates.
    """

    # Ensure both arrays are sorted by geo, gender, year, and age
    forecasted_rates = forecasted_rates[np.lexsort((forecasted_rates[:, 3], forecasted_rates[:, 2], forecasted_rates[:, 1], forecasted_rates[:, 0]))]
    actual_rates = actual_rates[np.lexsort((actual_rates[:, 3], actual_rates[:, 2], actual_rates[:, 1], actual_rates[:, 0]))]

    # Find common geo/gender/year/age combinations between forecasted and actual rates
    common_keys = set(map(tuple, forecasted_rates[:, :4])) & set(map(tuple, actual_rates[:, :4]))

    # Filter both forecasted and actual rates based on common combinations
    filtered_forecasted = np.array([row for row in forecasted_rates if tuple(row[:4]) in common_keys])
    filtered_actual = np.array([row for row in actual_rates if tuple(row[:4]) in common_keys])

    # Extract the forecasted and actual mortality rates
    forecasted_values = filtered_forecasted[:, 4]
    actual_values = filtered_actual[:, 4]
    
    # Calculate the overall MSE
    overall_mse = np.mean((forecasted_values - actual_values) ** 2)

    # Filter for states and countries
    states_mask = np.isin(filtered_forecasted[:, 0], range(0, 50))
    countries_mask = np.isin(filtered_forecasted[:, 0], range(51, 87))

    # Calculate MSE for states
    states_forecasted_values = filtered_forecasted[states_mask, 4].astype(float)
    states_actual_values = filtered_actual[states_mask, 4].astype(float)
    states_mse = np.mean((states_forecasted_values - states_actual_values) ** 2)

    # Calculate MSE for countries
    countries_forecasted_values = filtered_forecasted[countries_mask, 4].astype(float)
    countries_actual_values = filtered_actual[countries_mask, 4].astype(float)
    countries_mse = np.mean((countries_forecasted_values - countries_actual_values) ** 2)
    
    return {
        'overall_mse': overall_mse,
        'states_mse': states_mse,
        'countries_mse': countries_mse
    }

In [129]:
def run_lc_model(train_data, test_data):
    lc_output = lee_carter_geo_gender(train_data)
    predictions = lee_carter_forecast(lc_output, h=10, start_year=2006, ages=range(0, 100))
    test_mse = calculate_mse(predictions, test_data)

    return test_mse

In [130]:
lc_results = run_lc_model(train_data=training_data, test_data=test_data)

Skipping Geo: 74.0, Gender: 1.0 due to NaN or infinite values in m_x


In [131]:
lc_results

{'overall_mse': 0.323552848214168,
 'states_mse': 0.00019604996981467955,
 'countries_mse': 0.8015493675903409}

### Generate Table 1: Training and Test MSEs
This table will document average MSEs (for states alone, countries alone, and total) over 5 training runs with each model (LC, deep learning seperate, deep learning joint)

In [35]:
def compare_models(num_iterations):
    results = []
    for i in range(num_iterations):
        lc = run_lc_model(train_data=training_data, test_data=test_data)
        state_only = run_deep_model(dataset_train=state_dataset_train, dataset_test=state_dataset_test, geo_dim=state_geo_dim)
        country_only = run_deep_model(dataset_train=country_dataset_train, dataset_test=country_dataset_test, geo_dim=country_geo_dim)
        results.append((lc, state_only, country_only))
        print(f"Loop {i}: lc {results[i][0]} & state only {results[i][1]} & country only {results[i][2]}")

    return results
        

In [36]:
comparison_results = compare_models(num_iterations=5)

/var/folders/1z/wn6shwbs4_9gcwtsrgz8v5vc0000gn/T/ipykernel_41906/1271048323.py:24: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  m_x[i,j] = state_gender_data[mask, 4]


KeyboardInterrupt: 

In [15]:
comp_results_np = np.array(comparison_results)
comp_results_np

NameError: name 'comparison_results' is not defined